In [43]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [132]:
glucose = pd.read_csv('/home/v-tyan/tsf_ideas_real/data/T1D/552_glucose.csv')
bolus = pd.read_csv('/home/v-tyan/tsf_ideas_real/data/T1D/552_bolus.csv')
meal = pd.read_csv('/home/v-tyan/tsf_ideas_real/data/T1D/552_meal.csv')

In [135]:
bolus

,Unnamed: 0,ts_begin,ts_end,type,dose,bwz_carb_input
0,0,325.483333,16-04-2025 16:42:34,normal,2.10,NaN
1,1,389.883333,16-04-2025 17:46:58,normal,1.85,NaN
2,2,396.583333,16-04-2025 17:53:40,normal,1.50,NaN
3,3,441.200000,16-04-2025 18:38:17,normal,4.20,NaN
4,4,518.950000,16-04-2025 19:56:02,normal,1.70,NaN
...,...,...,...,...,...,...
429,97,74960.283333,07-06-2025 12:37:22,normal,6.90,NaN
430,98,75139.916667,07-06-2025 15:37:00,normal,6.60,NaN
431,99,75216.166667,07-06-2025 16:53:15,normal,2.00,NaN
432,100,75357.583333,07-06-2025 19:14:40,normal,1.80,NaN


In [136]:
np.sum(glucose.ts.diff() > 50)

27

In [137]:
interval_ens = glucose.ts[glucose.ts.diff() > 50] - glucose.ts.diff()[glucose.ts.diff() > 50]

In [138]:
interval_sts = glucose.ts[glucose.ts.diff() > 50]

In [139]:
interval_sts

869       4528.000000
1514      7918.216667
2074     10783.383333
2216     11883.450000
2465     13498.366667
2728     14918.483333
3248     17683.650000
3390     18518.700000
3827     22013.466667
4039     23168.550000
4320     24663.616667
4604     26363.750000
5451     33241.766667
5521     33767.800000
5955     36092.933333
6636     39588.150000
7086     43489.766667
8003     48220.050000
8760     53362.600000
8955     54457.666667
9070     55437.150000
9273     56567.233333
9757     59097.383333
10335    69062.400000
10407    69482.433333
11010    72932.633333
11378    74907.766667
Name: ts, dtype: float64

In [140]:
interval_ens

869       4380.250000
1514      7773.216667
2074     10718.383333
2216     11488.433333
2465     13138.533333
2728     14838.483333
3248     17533.616667
3390     18388.700000
3827     20718.816667
4039     23083.550000
4320     24568.616667
4604     26093.716667
5451     30634.000000
5521     33586.783333
5955     35947.933333
6636     39518.150000
7086     41833.283333
8003     48130.050000
8760     52110.283333
8955     54332.666667
9070     55027.700000
9273     56447.233333
9757     58997.383333
10335    61982.566667
10407    69417.416667
11010    72522.616667
11378    74797.750000
Name: ts, dtype: float64

In [141]:
basetimes = []

st = 0
for en, st_ in zip(interval_ens, interval_sts):
    print(f'[{st:.2f}, {en:.2f})')
    basetimes.append(np.arange(st, en, 5))
    st = st_
en = glucose.ts[len(glucose) - 1] + 1
print(f'[{st:.2f}, {en:.2f})')
basetimes.append(np.arange(st, en, 5))

[0.00, 4380.25)
[4528.00, 7773.22)
[7918.22, 10718.38)
[10783.38, 11488.43)
[11883.45, 13138.53)
[13498.37, 14838.48)
[14918.48, 17533.62)
[17683.65, 18388.70)
[18518.70, 20718.82)
[22013.47, 23083.55)
[23168.55, 24568.62)
[24663.62, 26093.72)
[26363.75, 30634.00)
[33241.77, 33586.78)
[33767.80, 35947.93)
[36092.93, 39518.15)
[39588.15, 41833.28)
[43489.77, 48130.05)
[48220.05, 52110.28)
[53362.60, 54332.67)
[54457.67, 55027.70)
[55437.15, 56447.23)
[56567.23, 58997.38)
[59097.38, 61982.57)
[69062.40, 69417.42)
[69482.43, 72522.62)
[72932.63, 74797.75)
[74907.77, 75233.77)


In [142]:
glucoses = []
boluss = []
meals = []

for bt in basetimes:
    gl = np.zeros_like(bt)
    bl = np.zeros_like(bt)
    ml = np.zeros_like(bt)

    start_time = bt[0]
    end_time = bt[-1]
    
    gll = glucose[glucose.ts >= start_time][glucose.ts <= end_time + 1]
    gl[((gll.ts - start_time) / 5 + 0.1).astype(int)] = gll.value
    for idx, v in enumerate(gl):
        if v < 1:
            gl[idx] = gl[idx - 1]
    
    bll = bolus[bolus.ts_begin >= start_time][bolus.ts_begin <= end_time + 1]
    bl[((bll.ts_begin - start_time) / 5 + 0.1).astype(int)] = bll.dose

    mll = meal[meal.ts >= start_time][meal.ts <= end_time + 1]
    ml[((mll.ts - start_time) / 5 + 0.1).astype(int)] = mll.carbs

    glucoses.append(gl)
    boluss.append(bl)
    meals.append(ml)

ipykernel_launcher:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
ipykernel_launcher:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
ipykernel_launcher:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
ipykernel_launcher:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
ipykernel_launcher:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
ipykernel_launcher:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
ipykernel_launcher:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
ipykernel_launcher:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
ipykernel_launcher:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
ipykernel_launcher:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
ipykernel_

In [143]:
np.savez('../data/T1D/data', basetimes=basetimes, glucoses=glucoses, boluss=boluss, meals=meals)

/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [99]:
data = np.load('../data/T1D/data.npz', allow_pickle=True)

In [100]:
data.files

['basetimes', 'glucoses', 'boluss', 'meals']

In [102]:
basetimes = data['basetimes']
glucoses = data['glucoses']
boluss = data['boluss']
meals = data['meals']

In [104]:
sum([len(bt) for bt in basetimes])

14926